# 4元数クラス

## クラス定義

4元数クラス:

* r-part (実部), i-part, j-part, k-part の4つの実数からなる。
    $$ 2-3i+j+5k$$
* 足し算と掛け算ができる。
* 表示機能がある。

In [0]:
import re
import math

class Quaternion:
    '''Quaternion class'''
    
    parts = ['r', 'i', 'j', 'k']
    
    mul_table = {
        'rr': 'r', 'ri': 'i', 'rj': 'j', 'rk': 'k',
        'ir': 'i', 'ii': '-r','ij': 'k', 'ik': '-j',
        'jr': 'j', 'ji': '-k','jj': '-r','jk': 'i',
        'kr': 'k', 'ki': 'j', 'kj': '-i','kk': '-r'
    }
     
    def __init__(self, vec = (0,0,0,0)): 
        self.coeffs = {'r':0, 'i':0, 'j':0, 'k':0}
        for i in range(len(self.parts)):
            self.coeffs[self.parts[i]] = vec[i]
                    
    def __add__(self, q):
        vec = [self.coeffs[p]+q.coeffs[p] for p in self.parts]
        return type(self)(vec)
    
    def show(self):
        mylist = [ str(self.coeffs[p]) + p for p in self.parts ]
        mystr = '+'.join( mylist )
        mystr = mystr.replace('r', '')

        mystr = re.sub(r'^0', '', mystr)                    # 0+2i --> +2i
        mystr = re.sub(r'([+-])1([ijk])', r'\1\2', mystr)   # 1+1i --> 1+i
        mystr = re.sub(r'[+]0[ijk]', '', mystr)             # 2+0i+3j --> 2+3j 
        mystr = re.sub(r'[+]([-+])', r'\1', mystr)          # 1+-3i --> 1-3i, 1++3i --> 1+3i
        mystr = re.sub(r'^[+]|[+]$', '', mystr)             # +2i --> 2i
        return mystr if mystr != '' else '0'
    #---
    def __mul__(self, q):
        result = type(self)()
        #---
        nonzeros = [(p,v) for p,v in self.coeffs.items() if v != 0]   # 係数がゼロでないパートの数を数える。
        if len(nonzeros) == 0:
            return result
        elif len(nonzeros) > 1:
            for p in self.parts:
                mono = type(self)()
                mono.coeffs[p] = self.coeffs[p]
                result += mono*q
        else:                                                         # ノンゼロパートが1つ、つまり単項式なら、掛け算実行。
            sp, sv = nonzeros[0][0], nonzeros[0][1]
            for qp, qv in q.coeffs.items():
                cc = sv*qv
                pp = self.mul_table[sp+qp]
                ppp = re.sub(r'[-]([rijk])', r'\1', pp)
                if pp != ppp:
                    cc = cc*(-1)
                    pp = ppp
                result.coeffs[pp] += cc
        return result
    
    def conjugate(self):
        result = type(self)()
        for p in self.parts:
            sign = -1 if p != 'r' else 1
            result.coeffs[p] = self.coeffs[p]*sign
        return result
    
    def norm(self):
        val = (self*(self.conjugate())).coeffs['r']
        return math.sqrt(val)
    
    def inverse(self):
        conj = self.conjugate()
        nrm_sq = (self*conj).coeffs['r']
        return type(self)([1/nrm_sq,0,0,0])*conj        
    #---    

In [0]:
q = Quaternion([1,2,3,4])
print(q.coeffs)
print(type(q.coeffs))

print(q.coeffs['i'])
print(q.show())

q1 = Quaternion([-2,0,1,5])
print((q*q1).show())

print(q.conjugate().show())
print(q.norm())

{'r': 1, 'i': 2, 'j': 3, 'k': 4}
<class 'dict'>
2
1+2i+3j+4k
-25+7i-15j-k
1-2i-3j-4k
5.477225575051661


## 3次元空間の回転運動

$\mathbb{R}^{3}$ において、

「原点を通る単位ベクトル $\vec{u}$ を回転軸として，右ねじの方向に角 $\theta$ だけ回転させる」
    
という回転運動を $R_{\vec{u},\theta}$ と表す。

ベクトル $R_{\vec{u},\theta}(\vec{x})$ は四元数を使って次のように表現できる。

4元数 $q$ を
$$
\displaystyle q=\cos {\theta \over 2} + u \sin {\theta \over 2}
~~~~~(\text{ここで}，u = iu_{1}+ju_{2}+ku_{3}).
$$
として,
$$
R_{\vec{u},\theta}(\vec{x}) = q \,\vec{x}\, \overline {q}.
$$



In [0]:
u = [0,1,0]
t = math.pi/2

uq = Quaternion([0]+u)
q = Quaternion([math.cos(t/2),0,0,0])+uq*Quaternion([math.sin(t/2),0,0,0])

x = [3,0,0]
xq = Quaternion([0]+x)

yq = q*xq*(q.conjugate())
print(yq.show())

.0+4.440892098500626e-16i+0.0j-3.0k


## テスト

In [0]:
from random import randrange

### メソッド ```__add__``` のテスト

In [0]:
vec_samples = [
    [[1,1,1,1], [-2,0,1,3], [-1,1,2,4]],
    [[1,0,0,0], [0,0,1,0], [1,0,1,0]],
    [[1,0,-1,0], [-1,0,1,0], [0,0,0,0]]
]
samples = [ [Quaternion(v) for v in vs] for vs in vec_samples ]

for s in samples:
    assert (s[0]+s[1]).coeffs == s[2].coeffs, 'FAILED'

### メソッド ```show``` のテスト

In [0]:
vec = [1,2,3,4]
q = Quaternion(vec)
print(q.show())
assert q.show() == '1+2i+3j+4k', 'FAILED'

In [0]:
data = ( 
        ([1,2,3,4], '1+2i+3j+4k'),
        ([0,1,1,1], 'i+j+k'),
        ([1,1,1,1], '1+i+j+k'),
        ([-1,0,3,1], '-1+3j+k'),
        ([1,-1,1,1], '1-i+j+k'),
        ([0,-2,1,-3], '-2i+j-3k'),
        ([0,0,0,0], '0'),
        ([0,1,0,0], 'i'),
        ([101,10,0,-21], '101+10i-21k'),
        )

for p in data:
    q = Quaternion(p[0])
    try:
        assert(q.show() == p[1])
    except AssertionError:
        print('FAILED  q = {}, q.show()={}'.format(q.coeffs, q.show()))
        break
else: print('PASSED') 

In [0]:
qi, qj, qk = Quaternion((0,1,0,0)), Quaternion((0,0,1,0)), Quaternion((0,0,0,1))
(qi.show(), qj.show(), qk.show())

### メソッド ```__mul__``` のテスト

In [0]:
data = []
for k in range(100):
    vec1 = [randrange(20)*((-1)**(randrange(2))) for k in range(4)]
    vec2 = [randrange(20)*((-1)**(randrange(2))) for k in range(4)]
    
    prdt = [
        vec1[0]*vec2[0]-vec1[1]*vec2[1]-vec1[2]*vec2[2]-vec1[3]*vec2[3],
        vec1[0]*vec2[1]+vec1[1]*vec2[0]+vec1[2]*vec2[3]-vec1[3]*vec2[2],
        vec1[0]*vec2[2]-vec1[1]*vec2[3]+vec1[2]*vec2[0]+vec1[3]*vec2[1],
        vec1[0]*vec2[3]+vec1[1]*vec2[2]-vec1[2]*vec2[1]+vec1[3]*vec2[0]        
    ]
    data.append((vec1, vec2, prdt))
    
for p in data:
    q1xq2 = Quaternion(p[0])*Quaternion(p[1])
    qp = Quaternion(p[2])

    try:
        assert q1xq2.coeffs == qp.coeffs
    except AssertionError:
        print('FAILED  q1xq2 = {}, qp={}'.format(q1xq2.show(), qp.show()))
        break
else: print('PASSED')

PASSED


### メソッド conjugate のテスト

In [0]:
data = []
for k in range(100):
    vec = [randrange(20)*((-1)**(randrange(2))) for k in range(4)]
    
    conj = [vec[0], vec[1]*(-1), vec[2]*(-1), vec[3]*(-1)]
            
    data.append((vec, conj))
    
for p in data:
    q = Quaternion(p[0])
    qc = Quaternion(p[1])

    try:
        assert q.conjugate().coeffs == qc.coeffs
    except AssertionError:
        print('FAILED  q = {} --> q.conjugate() = {}, qc={}'.format(q.show(), q.conjugate().show(), qc.show()))
        break
else: print('PASSED')

PASSED


### メソッド norm のテスト

In [0]:
data = []
for k in range(100):
    vec = [randrange(20)*((-1)**(randrange(2))) for k in range(4)]
    
    norm_sq = vec[0]**2+vec[1]**2+vec[2]**2+vec[3]**2
            
    data.append((vec, norm_sq))
    
for p in data:
    q = Quaternion(p[0])
    ns = p[1]

    try:
        assert round(q.norm()**2) == ns
#        assert q.norm() == math.sqrt(ns)
    except AssertionError:
        print('FAILED  q = {} --> q.norm()**2 = {}, ns={}'.format(q.show(), q.norm()**2, ns))
        break
else: print('PASSED')

PASSED


### メソッド inverse のテスト

In [0]:
data = [[1,0,0,0], [0,1,0,0], [1,1,0,0], [1,1,1,1], [-1,0,1,0]]
for k in range(10):
    vec = [randrange(20)*((-1)**(randrange(2))) for k in range(4)]
    data.append(vec)
    
for vec in data:
    q = Quaternion(vec)
    i = q*(q.inverse())
    for p,v in i.coeffs.items():
        i.coeffs[p] = round(v)
    
    try:
        assert i.show() == '1'
    except AssertionError:
        print('FAILED  q = {} --> q*(q.inverse) = {}'.format(q.show(), i.show()))
        break
else: print('PASSED')

PASSED


## `unittest` ライブラリを使う。

In [0]:
import unittest
from IPython.display import Markdown, display

def run_check(test_class):
    t = test_class
    try:
        t.check()
    except t.failureException as e:
        display(Markdown('**<span style="color: red;">FAILED</span>**'))
        print('Hint:',  e)
    else: display(Markdown('**<span style="color: green;">PASSED</span>**'))

In [0]:
class add_test(unittest.TestCase):
    vecs = [(0,0,0,0), (1,0,0,0), (0,1,0,0), (0,0,1,0), (0,0,0,1), (1,-2,-1,0)]
    pairs = ( ((0,0), (0,0,0,0)),
            ((0,1), vecs[0]),
            ((0,5), vecs[5]),
            ((3,5), (1,-2,0,0))
             )
       
    def check(self):
        for p, v in self.pairs:
            s = Quaternion(self.vecs[p[0]]) + Quaternion(self.vecs[p[1]])
            self.assertEqual(s.coeffs, v)
            
run_check(add_test())

In [0]:
class show_test(unittest.TestCase):
    pairs = ( 
        ([1,2,3,4],'1+2i+3j+4k'),
        ([0,1,1,1], 'i+j+k'),
        ([1,1,1,1],'1+i+j+k'),
        ([-1,0,3,1],'-1+3j+k'),
        ([1,-1,1,1],'1-i+j+k'),
        ([0,-2,1,-3],'-2i+j-3k'),
        ([0,0,0,0],'0'),
        ([0,1,0,0],'i'),
        ([101,10,0,-21],'101+10i-21k'),
        )
    
    def check(self):
        for v, s in self.pairs:
            q = Quaternion(v)
            self.assertEqual(q.show(), s)
            
run_check(show_test())

## 退避場所

In [0]:
class Quaternion:
    '''Quaternion class'''
    
    parts = ['r', 'i', 'j', 'k']
    
    def __init__(self, vec = (0,0,0,0)):
        self.coeffs = {'r':0, 'i':0, 'j':0, 'k':0}
        for i in range(len(self.parts)):
            self.coeffs[self.parts[i]] = vec[i]
            
#    def get_coeff(self, p):
#        return self.coeffs[p]
        
    def __add__(self, q):
        pass
#        vec = ([self.get_coeff(p) + q.get_coeff(p) for p in self.parts])
#        return type(self)(vec)
        
    def __product__(self, q):
        pass
    
    def show(self):
        pass 

In [0]:
import re

class Quaternion:
    '''Quaternion class'''
    
    parts = ['r', 'i', 'j', 'k']
    
    def __init__(self, vec = (0,0,0,0)):
        self.coeffs = {'r':0, 'i':0, 'j':0, 'k':0}
        for i in range(len(self.parts)):
            self.coeffs[self.parts[i]] = vec[i]
    
    def __add__(self, q):
        if type(q) != type(self):
            raise ValueError("Can't add {}".format(type(q)))
        vec = tuple([self.get_coeff(p) + q.get_coeff(p) for p in self.parts])
        return type(self)(vec)
    
    def get_coeff(self, p):
        return self.coeffs[p]
    
    def show(self):
        mstr = '+'.join([str(self.get_coeff(p)) + p for p in self.parts])
        mstr = mstr.replace('r','')                      # 2r+3i --> 2+3i
        mstr = re.sub(r'1([ijk])', r'\1', mstr)          # 1+1i --> 1+i
        mstr = re.sub(r'^0', '', mstr)   # 0+2i --> +2i, 2+0i+3j --> 2++3j
        mstr = re.sub(r'[+]0[ijk]', '', mstr)
        mstr = re.sub(r'[+]([-+])',r'\1', mstr)         # 1+-3i --> 1-3i, 1++3i --> 1+3i
        mstr = re.sub(r'^[+]|[+]$','', mstr)             # +2i --> 2i
        return mstr if mstr != '' else '0'

In [0]:
import re

class Quaternion:
    '''Quaternion class'''
    
    parts = ['r', 'i', 'j', 'k']
    
    def __init__(self, vec = (0,0,0,0)): 
        self.coeffs = {'r':0, 'i':0, 'j':0, 'k':0}
        for i in range(len(self.parts)):
            self.coeffs[self.parts[i]] = vec[i]
                    
    def __add__(self, q):
        rc = self.coeffs['r']+q.coeffs['r']
        ic = self.coeffs['i']+q.coeffs['i']    
        jc = self.coeffs['j']+q.coeffs['j']    
        kc = self.coeffs['k']+q.coeffs['k']
        return type(self)([rc, ic, jc, kc])
    
    def show(self):
        mylist = [ str(self.coeffs[p]) + p for p in self.parts ]
        mystr = '+'.join( mylist )
        mystr = mystr.replace('r', '')

        mystr = re.sub(r'^0', '', mystr)                    # 0+2i --> +2i
        mystr = re.sub(r'([+-])1([ijk])', r'\1\2', mystr)   # 1+1i --> 1+i
        mystr = re.sub(r'[+]0[ijk]', '', mystr)             # 2+0i+3j --> 2+3j 
        mystr = re.sub(r'[+]([-+])', r'\1', mystr)          # 1+-3i --> 1-3i, 1++3i --> 1+3i
        mystr = re.sub(r'^[+]|[+]$', '', mystr)             # +2i --> 2i
        return mystr if mystr != '' else '0'
    #---
    def times(self, q): 
        for p in self.coeffs:
            #print(p + str(self.coeffs[p]) )
            for p2 in q.coeffs:
                cc = self.coeffs[p]*(q.coeffs[p2])
                vec = [0,0,0,0]
                if p == 'r':
                    pp = p2
                elif p == 'i':
                    if p2 == 'r':
                        pp = 'i'
                        vec[1] = cc
                    elif p2 == 'i':
                        pp = 'r'
                        cc = cc*(-1)
                        vec[0] = cc
                    elif p2 == 'j':
                        pp = 'k'
                        vec[3] = cc
                    else:
                        pp = 'j'
                        cc = cc*(-1)
                        vec[2] = cc
                    quat = Quaternion(vec)
                elif p == 'j':
                    if p2 == 'r':
                        pp = 'j'
                    elif p2 == 'i':
                        pp = 'k'
                        cc = cc*(-1)
                    elif p2 == 'j':
                        pp = 'r'
                        cc = cc*(-1)
                        vec[0] = cc
                    else:
                        pp = 'i'
                        vec[1] = cc
                    quat = Quaternion(vec)
                elif p == 'k':
                    if p2 == 'r':
                        pp = 'k'
                    elif p2 == 'i':
                        pp = 'j'
                    elif p2 == 'j':
                        pp = 'i'
                        cc = cc*(-1)
                        vec[1] = cc
                    else:
                        pp = 'r'
                        cc = cc*(-1)
                        vec[0] = cc
                    quat = Quaternion(vec)

In [0]:
    def conjugate(self):
        result = type(self)()
        for p in self.parts:
            sign = -1 if p != 'r' else 1
            result.coeffs[p] = self.coeffs[p]*sign
        return result

    def norm(self):
        val = (self*(self.conjugate())).coeffs['r']
        return math.sqrt(val)
    
    def inverse(self):
        conj = self.conjugate()
        nrm_sq = (self*conj).coeffs['r']
        return type(self)([1/nrm_sq,0,0,0])*conj

